In [1]:
import pandas as pd
import sqlalchemy
from components.indicators import beta, EMA

engine = sqlalchemy.create_engine('sqlite:///stock_etf.db')
data = pd.read_sql_query("SELECT * FROM stockData", engine, parse_dates="Date")

In [10]:
any_pairs = '''APH/AX BYD/AX BRO/HOG VSH/ABCB AGRO/ADEA BYD/ACIW AFL/JNPR BKR/ACIW ZTO/ACIW VSH/AGNC HAS/BXP HAS/COLB 
BWA/LAZ NEO/AB SKT/M PAYX/EXR NS/ENV HP/SPR UAL/SLG FORM/SLG USAC/AMKR CF/BHP AU/CX BHP/NEM CENX/MT CF/EMN CLF/AGI 
CMC/X CSTM/CENX FMC/AEM FMC/EMN FNV/CE FNV/RIO FNV/RPM AA/GPRE HBM/CX HUN/CENX SAVE/AGI AAP/ST SNY/AMKR RTO/AMKR 
VICI/AMKR YUMC/AX SLB/AY ZTO/AY CAH/AYX HAL/AYX CCI/AZTA HSY/AZTA KMX/AZTA NKE/AZTA RVTY/AZTA GIS/BJ MDLZ/BJ
YUMC/BTU WPM/BTU BCE/CLF BMY/CLF RMD/COHR DINO/CRNX RMD/CRSP AEE/CRUS CRUS/AMGN CRUS/FDX BMY/CUBE
'''

In [14]:
positive_statistics = '''AB/INCY AB/WPM ACHC/NTR AX/USFD AXNX/ROST AXNX/SNY AXNX/UBS AZTA/AOS AZTA/ATO AZTA/AZN 
AZTA/BABA AZTA/BCE AZTA/BG AZTA/BUD AZTA/CAG AZTA/CCEP AZTA/PPG BL/AEE BL/BMY BL/CMCSA BL/GMAB BL/XEL 
BLMN/DVN BRKR/AKAM BRKR/NRG BTU/HRL CARG/DAL CGNX/ZTO CIEN/BAX CIEN/DLTR CIEN/UHS CIEN/WPM CLF/GLPI CLF/IBN CLF/NGG
CMC/ES CMC/ETR CMC/EVRG CMC/FE CMC/NRG CMC/OMC CMC/RVTY CMC/WAB CROX/AWK CROX/BR CROX/BX CROX/EFX CROX/FDX CROX/LH
CRSP/FDX CRUS/BAX CRUS/BHP CRUS/PRU CVNA/TROW
'''

In [15]:
dividend_positive = '''AEP/AKAM AEP/CHKP AEP/CNC ADM/CRUS APD/CRUS AEP/FLT AEP/FSLR AEP/VRSN ARCC/MGM ARCC/SQ 
ARCC/ZG BBY/AKAM BBY/ALNY BBY/EXAS BBY/FSLR BBY/KEYS BBY/MNST BMO/FSLR BMO/INCY BMO/PSTG BMO/SPLK BNS/HSIC 
BNS/MNST CCI/AKAM CCI/AZTA CCI/ALNY CCI/BMRN CCI/TTWO CCI/VRSN CCI/EXAS CM/DXCM CM/MNST CM/PSTG CM/PYPL CME/ADSK 
CME/ALNY CME/ANSS CME/BIIB CME/CHKP CME/DXCM CME/FFIV CME/FIVE CME/FLT CME/FSLR CME/ILMN CME/PTC CME/PODD 
CNQ/DXCM CNQ/FIVE CNQ/HSIC CNQ/INCY CNQ/KEYS CNQ/MGM CNQ/MRNA CNQ/PODD CPT/BIDU CPT/BIIB CPT/FSLR CPT/EXAS 
CPT/DXCM CPT/VRSN DUK/AKAM DUK/ALNY DUK/CHKP DUK/MNST DUK/PDD DUK/SQ DUK/PYPL DUK/VRSN EIX/PSTG EIX/FIVE 
EIX/GMAB EPD/DOCU EPD/HSIC EPD/INCY EPD/MGM EQR/CHKP EQR/FFIV EQR/FWONK EQR/GMAB EQR/KEYS EQR/MNST EQR/PODD 
EQR/SPLK ES/AKAM ES/BIDU ES/CHKP ES/FFIV ESS/ANSS ESS/BIIB ESS/EXPE ESS/FLT ESS/FSLR ESS/NBIX ESS/VRSN 
EVRG/AKAM EVRG/KEYS EXC/DXCM EXC/GMAB EXC/INCY EXC/MGM EXC/PSTG EXR/BIIB EXR/FSLR EXR/KEYS FANG/CBRE 
FANG/CNC FANG/DLTR FANG/FIVE FANG/HSIC FANG/INCY FANG/WAT FE/HSIC FE/MNST FTS/MNST IPG/PSTG K/HSIC
KMB/AKAM KMB/CHKP KMB/DXCM KMB/FSLR KMB/KEYS LAMR/AKAM LAMR/KEYS LYB/CHKP MAA/ALNY MAA/FSLR MPLX/MGM
MPLX/PFGC MPLX/PSTG MS/AKAM OKE/HSIC OKE/INCY PNC/CHKP PNC/EXPE PNC/FLT PNC/TTWO PNC/ZG PRU/ANSS PRU/AER
PRU/DXCM PRU/EXPE PSA/ALNY PSA/BIIB PSA/FSLR RF/MGM RF/MNST RF/PSTG RIO/AKAM RIO/ALNY RY/AKAM RY/CHKP 
RY/FSLR RY/KEYS RY/PSTG SLF/DXCM SLF/EXAS SLF/FSLR SLF/FTNT SLF/FWONK SLF/MNST SLF/PSTG SO/MNST SPG/HOLX
STLA/GMAB STLA/INCY STLA/KMX SU/INCY TAK/CCL TD/MGM TD/EXPE TROW/AKAM TROW/ALNY TROW/FSLR TROW/HSIC 
TROW/KMX TROW/LYV TROW/SPLK TRP/BMRN USB/EXPE VZ/PYPL '''

In [5]:
def ratios(pairs: str):
    pair_list = pairs.split()
    ratios = pd.DataFrame()
    for pair in pair_list:
        stock = pair.split('/')
        ratio = pd.DataFrame({
            f'{stock[0]}/{stock[1]}': data[f'{stock[0]}'] / data[f'{stock[1]}']
        })
        ratios = pd.concat([ratios, ratio], axis=1)
    return ratios

In [16]:
any_pair_ratios = ratios(any_pairs)
positive_statistics_ratios = ratios(positive_statistics)
dividend_positive_ratios = ratios(dividend_positive)

In [6]:
def ratio_beta(pairs: str):
    pair_list = pairs.split()
    betas = pd.DataFrame()
    
    for pair in pair_list:
        stock = pair.split('/')
        long_beta = beta(data[f'{stock[0]}'], f'{stock[0]}').round(2)
        short_beta = beta(data[f'{stock[1]}'], f'{stock[1]}').round(2)
        
        beta_value = long_beta / short_beta
        
        beta_ratio = pd.DataFrame({
            f'{stock[0]}/{stock[1]}': beta_value.round(2)
        }, index=[0])
        betas = pd.concat([betas, beta_ratio], axis=1)
    return betas

In [17]:
any_pair_beta = ratio_beta(any_pairs)
positive_statistics_beta = ratio_beta(positive_statistics)
dividend_positive_beta = ratio_beta(dividend_positive)

In [7]:
def pct_returns(given_df: pd.DataFrame):
    c_c_returns = ((given_df - given_df.shift(1)) / given_df.shift(1))*100
    return c_c_returns

In [18]:
any_pair_cc_returns = pct_returns(any_pair_ratios)
positive_statistics_cc_returns = pct_returns(positive_statistics_ratios)
dividend_positive_cc_returns = pct_returns(dividend_positive_ratios)

In [32]:
def watchlist(pairs: str, c_c_returns: pd.DataFrame, ratios: pd.DataFrame, ratio_beta: pd.DataFrame):
    pair_list = pairs.split()
    descriptive_statistics = pd.DataFrame()
    
    for pair in pair_list:
        stock = pair.split('/')
        
        pair_statistics = pd.DataFrame({
            pair: {
                'Long': stock[0],
                'Short': stock[1],
                'Beta': ratio_beta[pair].iloc[-1].round(2),
                'Ratio': ratios[pair].iloc[-1].round(2),
                'EMA20': EMA(ratios[pair], 20).iloc[-1].round(2),
                'EMA60': EMA(ratios[pair], 60).iloc[-1].round(2),
                'EMA200': EMA(ratios[pair], 200).iloc[-1].round(2),
                'Mean': c_c_returns[pair].mean().round(3),
                'STD': c_c_returns[pair].std().round(2),
                'Z_Score': ((c_c_returns[pair].iloc[-1] - c_c_returns[pair].mean()) / c_c_returns[pair].std()).round(2),
                'Kurtosis': c_c_returns[pair].kurtosis().round(2),
                'Skewness': c_c_returns[pair].skew().round(2),
                'Min': c_c_returns[pair].min().round(2),
                'Max': c_c_returns[pair].max().round(2)
                
            }
        })
        
        descriptive_statistics = pd.concat([descriptive_statistics, pair_statistics], axis=1)
    
    return descriptive_statistics.T

In [33]:
any_pairs_watchlist = watchlist(pairs=any_pairs, c_c_returns=any_pair_cc_returns, ratios=any_pair_ratios, ratio_beta=any_pair_beta)
positive_statistics_watchlist = watchlist(pairs=positive_statistics, c_c_returns=positive_statistics_cc_returns, ratios=positive_statistics_ratios, ratio_beta=positive_statistics_beta)
dividend_positive_watchlist = watchlist(pairs=dividend_positive, c_c_returns=dividend_positive_cc_returns, ratios=dividend_positive_ratios, ratio_beta=dividend_positive_beta)

In [34]:
any_pairs_watchlist.to_sql("Any_pairs_watchlist", engine, if_exists='replace', index=False)
positive_statistics_watchlist.to_sql("Pos_statistics_watchlist", engine, if_exists='replace', index=False)
dividend_positive_watchlist.to_sql("Div_positive_watchlist", engine, if_exists='replace', index=False)

181

In [41]:
pd.read_sql_query("""SELECT * FROM Div_positive_watchlist WHERE Beta > 1.5 
                  AND Ratio > EMA20 AND Ratio > EMA60 AND Ratio > EMA200
                  AND Z_Score < 0""", engine)

,Long,Short,Beta,Ratio,EMA20,EMA60,EMA200,Mean,STD,Z_Score,Kurtosis,Skewness,Min,Max
0,FANG,INCY,1.53,2.98,2.77,2.65,2.44,0.134,3.66,-0.47,12.57,0.19,-40.79,33.95
1,OKE,INCY,1.75,1.22,1.18,1.15,1.06,0.109,3.30,-0.12,15.68,0.17,-33.43,34.14
2,SPG,HOLX,1.52,1.98,1.96,1.90,1.69,-0.004,2.58,-0.28,31.40,1.81,-18.91,40.40
3,STLA,INCY,2.49,0.42,0.40,0.38,0.33,0.130,3.04,-0.58,5.48,0.30,-19.24,28.19
